<a href="https://colab.research.google.com/github/kousiknandy/pycolab/blob/main/SimpleWiki_Related.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import urllib3
from bs4 import BeautifulSoup

class WikiRelated:
  host = "https://simple.wikipedia.org"

  def __init__(self, url):
    self.url = url
    self.data = None

  def __call__(self):
    resp = urllib3.request("GET", self.host + self.url)
    if resp.status != 200: return None
    self.data = resp.data
    return resp.data

  def related(self):
    if not self.data: self()
    if not self.data: return
    soup = BeautifulSoup(self.data, "html.parser")
    h2 = soup.find(id="Related_pages")
    rl = h2.parent.next_siblings
    _ = next(rl)
    ul = next(rl)
    links = ul.find_all("a")
    for l in links: yield l["href"]


In [26]:
s = WikiRelated("/wiki/Computer_science")
list(s.related())

['/wiki/Computing',
 '/wiki/Formal_language',
 '/wiki/Turing_Award',
 '/wiki/Computer_jargon',
 '/wiki/List_of_words_about_computers']